# 1. Setup & Bus Data Load

Import required modules and packages.

In [ ]:
# import os so that environment variables can be accessed (for database password, etc.)
import os

# import mysql connector so that data can be pulled into the notebook directly from the database
import mysql.connector

# import pandas and numpy for data analysis
import pandas as pd
import numpy as np

# import matplotlib & seaborn for plotting and visualisaion
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# import transform_data function for transforming data into segment format
from transform_data import transform_data

# import convert_timestamp for various timestamp conversion functions
import convert_timestamp

Set the max number of columns & rows to display.

In [ ]:
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 500)

Load data for a bus route from file.

In [ ]:
df = pd.read_csv('./data/route_15A.csv', sep=";", na_values=['\\N'])

Perform a check to see how many rows and columns are in the file.

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("Before any data cleaning, the CSV file contains", rows, "rows and", cols, "columns.")
print()

Print the first five lines of the dataframe.

In [ ]:
df.head(5)

# 2. Initial Checks on the Bus Data

## 2.1 Check for Duplicate Rows & Columns

In [ ]:
print()
print('Duplicate rows:', df.duplicated()[df.duplicated() == True].shape[0])
print('Duplicate columns:',df.columns.size - df.columns.unique().size)

There are no duplicate rows or columns in the bus data.

## 2.2 Check for Null/Empty Features

In [ ]:
df.describe().T

Features with a count of zero can be dropped as they contain no useful information:

In [ ]:
df = df.drop(columns=['tenderlot', 'suppressed_trip', 'justificationid_trip', 'passengers', 'passengersin', 'passengersout', \
                      'distance_leavetimes', 'note_leavetimes', 'note_vehicle'])

## 2.3 Assign Features as Continuous or Categorical

First check the data types of all rows after the file import.

In [ ]:
df.dtypes

Assign categorical and continous features, and update the type of all categorical features to 'category'.

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['datasource', 'dayofservice', 'tripid', 'lineid', 'routeid', 'direction', 'basin', \
                         'lastupdate_trip', 'note_trip', 'progrnumber', 'stoppointid', \
                          'suppressed_leavetimes', 'lastupdate_leavetimes']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

## 2.4 Check for Constant Categorical Features

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

Features with a count of one are constant and can be dropped. <br> **lineid** is constant for this subset of data, but is not constant for the full data set so will not be dropped at this stage.

In [ ]:
df = df.drop(columns=['datasource', 'basin'])

## 2.5 Check for Constant Continuous Features

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no constant continuous features so nothing needs to be dropped.

# 3. Further Analysis of Features

There are a number of features that can be dropped because they fall into at least one of the following categories:
- Features that don't provide much information
- Features that we won't be able to provide information on to the model

These features can be dropped:

In [ ]:
df = df.drop(columns=['lastupdate_trip', 'note_trip', 'suppressed_leavetimes', 'justificationid_leavetimes', \
                      'lastupdate_leavetimes','vehicleid', 'distance_vehicle', 'minutes_vehicle'])

# 4. Initial Checks for Missing Data

## 4.1 Categorical Features

Select the categorical features and print details:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction', 'progrnumber', 'stoppointid']].columns

# Print details for the categorical columns
df[categorical_columns].describe().T

There is a full count for all categorical features.

## 4.2 Continuous Features

Select the continuous features and print details:

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

# Print details for the continuous columns
df[continuous_columns].describe().T

There are some rows missing data for **actualtime_arr_trip** and **actualtime_dep_trip**. This will be reviewed if these features are used in the future, currently they are not carried across when data is transformed.

# 5. Transform the Bus Data

Bus data must be transformed so that each row of data holds information on one journey segment. This is done by calling the *transform_data* function:

In [ ]:
df_transformed = transform_data(df)

In [ ]:
df_transformed

## 5.1 Check for Missing Data

First re-assign the transformed data as continuous or categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns = df_transformed[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', \
                          'progrnumber_next', 'stoppointid_next']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_transformed[column] = df_transformed[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_transformed.select_dtypes(['float64', 'int64']).columns

Then check for missing data:

In [ ]:
# Print details for the categorical columns
df_transformed[categorical_columns].describe().T

In [ ]:
# Print details for the continuous columns
df_transformed[continuous_columns].describe().T

There are some rows with missing data, because the amount of missing rows is quite low, and because imputation would be difficult, these rows will be dropped.

## 5.2 Drop Rows with Missing Data

Drop rows where *stoppointid_first* or *stoppointid_next* is null:

In [ ]:
df_transformed = df_transformed[pd.notnull(df_transformed['stoppointid_first'])]

In [ ]:
df_transformed = df_transformed[pd.notnull(df_transformed['stoppointid_next'])]

# 6. Import Weather Data

Weather data is loaded from the database:

In [ ]:
# open connection
connection = mysql.connector.connect(host=os.environ['DBHOST'], user=os.environ['DBUSER'], \
        password=os.environ['DBPASS'], db='db_raw_data')

# SQL query 
sql = "SELECT * FROM weather_data \
WHERE record_date BETWEEN CAST('2018-01-01' AS DATE) AND CAST('2019-01-01' AS DATE);"

# load into dataframe
df_weather = pd.read_sql(sql, connection)

## 6.1 Check for Duplicate Rows & Columns

In [ ]:
print()
print('Duplicate rows:', df_weather.duplicated()[df_weather.duplicated() == True].shape[0])
print('Duplicate columns:',df_weather.columns.size - df_weather.columns.unique().size)

There are no duplicate rows or columns so nothing needs to be dropped here.

## 6.2 Assign Features as Continuous or Categorical

First check the data types of all rows after the file import:

In [ ]:
df_weather.dtypes

Assign categorical and continuous features:

In [ ]:
# Select columns containing categorical data
categorical_columns = df_weather[['record_date', 'irain', 'itemp', 'iwb']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_weather.select_dtypes(['float64', 'int64']).columns

## 6.3 Check for Missing Data, Constant Features, etc.

In [ ]:
# Print details for the categorical columns
df_weather[categorical_columns].describe().T

**itemp** and **iwb** are constant columns so can be dropped.

In [ ]:
# Print details for the categorical columns
df_weather[continuous_columns].describe().T

Investigate rows with missing data for rain:

In [ ]:
# select all rows where irain is not 0
df_weather.loc[df_weather['irain'] != 0]

There are only two rows where irain is not zero, these rows correspond to missing values for rain.

In [ ]:
# select other rows around the missing values
df_weather[6220:6240]

Given that there is no rain for the rest of the day, and given the high (for Ireland) temperature on the day, I think it's safe to replace the missing rain values with 0.

I will then drop the feature **irain** as it provides no useful information.

## 6.4 Replace Missing Weather Data

In [ ]:
# replace rain with 0 where irain is not 0
df_weather['rain'].loc[df_weather['irain'] != 0] = 0

In [ ]:
# check that values are updated
df_weather.loc[df_weather['irain'] != 0]

## 6.5 Drop Constant Weather Features

In [ ]:
df_weather = df_weather.drop(columns=['irain', 'itemp', 'iwb'])

# 7. Combine Bus and Weather Data

## 7.1 Split Timestamp for Weather Data

To merge the data, timestamps must be split into month, day and hour.

New features are added using lambda functions:

In [ ]:
df_weather['month'] = df_weather.apply (lambda row: convert_timestamp.timestamp_to_month_weather(row['record_date']), axis=1)

In [ ]:
df_weather['day'] = df_weather.apply (lambda row: convert_timestamp.timestamp_to_day_weather(row['record_date']), axis=1)

In [ ]:
df_weather['hour'] = df_weather.apply (lambda row: convert_timestamp.timestamp_to_hour_weather(row['record_date']), axis=1)

New features are updated to be categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns = df_weather[['record_date', 'month', 'day', 'hour']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
df_weather

## 7.2 Split Timestamp for Bus Data

New features are added using lambda functions:

In [ ]:
df_transformed['month'] = df_transformed.apply (lambda row: convert_timestamp.timestamp_to_month_bus(row['dayofservice'], \
                                                                                   row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df_transformed['day'] = df_transformed.apply (lambda row: convert_timestamp.timestamp_to_day_bus(row['dayofservice'], \
                                                                               row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df_transformed['hour'] = df_transformed.apply (lambda row: convert_timestamp.timestamp_to_hour_bus(\
                                                                                row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df_transformed

## 7.3 Merge the Dataframes

In [ ]:
df_merged = pd.merge(df_transformed, df_weather,  how='left', left_on=['month','day', 'hour'],\
                     right_on = ['month','day', 'hour'])

Check that there are no rows missing weather data:

In [ ]:
df_merged[df_merged.rain.isnull()]

Update the data types for the new dataframe:

In [ ]:
# Select columns containing categorical data
categorical_columns = df_merged[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_merged[column] = df_merged[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_merged.select_dtypes(['float64', 'int64']).columns

In [ ]:
df_merged

# 8. Create New Features

## 8.1 Day of Week Feature

The new feature is added using a lambda function:

In [ ]:
df_merged['day_of_week'] = df_merged.apply (lambda row: convert_timestamp.timestamp_to_day_of_week(row['dayofservice']), axis=1)

## 8.2 Weekend/Weekday Feature

The new feature is added using a lambda function:

In [ ]:
df_merged['weekday'] = df_merged.apply (lambda row: convert_timestamp.timestamp_to_weekday_weekend(row['dayofservice']), axis=1)

## 8.3 Bank Holiday Feature

Make list of bank holidays for 2018 (based on https://www.officeholidays.com/countries/ireland/2018):

In [ ]:
holidays = ['2018-01-01 00:00:00', '2018-03-19 00:00:00', '2018-04-02 00:00:00', '2018-05-07 00:00:00', '2018-06-04 00:00:00',\
           '2018-08-06 00:00:00', '2018-10-29 00:00:00', '2018-12-25 00:00:00', '2018-12-26 00:00:00']

The new feature is added using a lambda function:

In [ ]:
df_merged['bank_holiday'] = df_merged.apply (lambda row: convert_timestamp.timestamp_to_bank_holiday(row['dayofservice'], \
                                                                                                      holidays), axis=1)

## 8.4 Time Difference Feature

The new feature is added using a lambda function:

In [ ]:
df_merged['time_diff'] = df_merged.apply (lambda row: row['actualtime_arr_stop_next'] - row['actualtime_arr_stop_first'],\
                                          axis=1)

## 8.5 Update Data Types for the Features

In [ ]:
# Select columns containing categorical data
categorical_columns = df_merged[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date', 'day_of_week', 'weekday', 'bank_holiday']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_merged[column] = df_merged[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_merged.select_dtypes(['float64', 'int64']).columns

In [ ]:
df_merged.dtypes

# 9. Data Integrity Checks

## 9.1 Time Difference Checks

Check for rows where the time difference is less than or equal to zero:

In [ ]:
df_merged.loc[df_merged['time_diff'] <= 0]

There are 167 rows that fail this test. These will be dropped:

In [ ]:
df_merged = df_merged.loc[df_merged['time_diff'] > 0]

## 9.2 Segment Checks

The current 15A route visits the following stops:

**Towards Ringsend:**

1105,1107,1108,1109,1110,1111,1112,1113,1114,1115,2437,1117,1118,1119,1120,1164,1165,1166,1167,1168,
1169,1170,1069,1070,1071,4528,1072,7577,1353,1354,7578,7582,0340,0350,0351,0352,0353,0354

**Towards Limekiln Ave:**

0395,0396,0397,0398,0399,0400,7581,1283,7579,1285,1016,1017,1018,1019,1020,1076,1077,1078,1079,1080,
1081,1082,1083,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1101,1102,1103,1104

We need to check that we have data for all stop combinations.

In [ ]:
# check if we have data for all stops going towards Ringsend
stops = [1105,1107,1108,1109,1110,1111,1112,1113,1114,1115,2437,1117,1118,1119,1120,1164,1165,1166,1167,1168,1169,1170,1069,\
         1070,1071,4528,1072,7577,1353,1354,7578,7582,340,350,351,352,353,354]

# loop through each combination of stops and print relevant data
for i in range(len(stops)-1):
    print(str(stops[i]) + " to " + str(stops[i+1]) + ": ")
    df_temp = df_merged.loc[(df_merged['stoppointid_first'] == stops[i]) & (df_merged['stoppointid_next'] == stops[i+1])]
    print(str(df_temp.shape[0]) + "\n")

In [ ]:
# check if we have data for all stops going towards Limekiln Ave
stops = [395,396,397,398,399,400,7581,1283,7579,1285,1016,1017,1018,1019,1020,1076,1077,1078,1079,1080, 1081,1082,1083,
         1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1101,1102,1103,1104]

# loop through each combination of stops and print relevant data
for i in range(len(stops)-1):
    print(str(stops[i]) + " to " + str(stops[i+1]) + ": ")
    df_temp = df_merged.loc[(df_merged['stoppointid_first'] == stops[i]) & (df_merged['stoppointid_next'] == stops[i+1])]
    print(str(df_temp.shape[0]) + "\n")

For the route going towards **Limekiln Ave.** we have data for all segments.

For the route going towards **Ringsend** we are missing data for the segment **1354 to 7578**.

We also have less data than usual for the following segments:

- **340 to 350**
- **350 to 351**
- **351 to 352**
- **352 to 353**

For these segments, we have approx. 3000 records compared with approx. 15000 for other segments.

The missing segment seems to be due to a removed stop - 7589. It is possible that other routes may cover the segment that we are missing data for, however we may need to use timetable or real time information for this segment. There is also the possibility of using times between 1354 and 7589 and 7589 and 7578 to approximate travel time, but we would need more info on where stop 7589 was to see if this would be valid to do.

# 10. Plot Features

## 10.1 Histograms for the Continuous Features

In [ ]:
df_merged[continuous_columns].hist(figsize=(30,30))

## 10.2 Box Plots for the Continuous Features

In [ ]:
for col in continuous_columns:
    f = df_merged[col].plot(kind='box', figsize=(10,5))
    plt.show()

## 10.3 Bar Charts for the Categorical Features

In [ ]:
# select the features to plot
# some categorical features will not be plotted here due to the high number of values e.g. bus stop no.
categorical_columns_plot = df_merged[['lineid', 'routeid', 'direction',  \
                          'month', 'day', 'hour', 'day_of_week', 'weekday', 'bank_holiday']].columns

In [ ]:
for column in categorical_columns_plot:
    f = df_merged[column].value_counts().plot(kind='bar', title=column, figsize=(8,6))
    plt.show()

## 10.4 Correlation Matrix for Continuous Features

In [ ]:
# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html
sns.set(style="white")

# Calculate correlation of all pairs of continuous features
corr = df_merged[continuous_columns].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))

# Generate a custom colormap - blue and red
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
            square=True, xticklabels=True, yticklabels=True,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
plt.yticks(rotation = 0)
plt.xticks(rotation = 90)

We are mainly interested here in the correlation between the weather data and *time_diff* or *actualtime_arr_stop_next*. The correlation for these features with the weather data is quite low.

Some of the weather features have very high correlation with each other e.g. temp with wetb and vappr with dewpt. The weather features we can use will be dependent on what is available in the weather forecast we receive. However, if all features are available in the forecast we may need to remove some of the features due to high correlation.

## 10.5 Correlations between the Continuous Features & the Target Feature

We have two options for our target feature:

- actualtime_arr_stop_next
- time_diff

I will plot scatter plots for each.

In [ ]:
# print a scatter plot for each continuous feature against actualtime_arr_stop_next
for col in continuous_columns:
    df_merged.plot.scatter(x=col, y='actualtime_arr_stop_next', c='purple')

In [ ]:
# print a scatter plot for each continuous feature against time_diff
for col in continuous_columns:
    df_merged.plot.scatter(x=col, y='time_diff', c='purple')

## 10.6 Correlations between the Categorical Features & the Target Feature

We have two options for our target feature:

- actualtime_arr_stop_next
- time_diff

I will plot box plots for each.

In [ ]:
for col in categorical_columns_plot:
    df_merged.boxplot(column=['actualtime_arr_stop_next'], by=[col], flierprops=flierprops, figsize=(8,5))

In [ ]:
for col in categorical_columns_plot:
    df_merged.boxplot(column=['time_diff'], by=[col], flierprops=flierprops, figsize=(8,5))

# 11. Deal with Outliers

To be investigated...

# 12. Feature Selection for the Initial Model

For an initial model, I will use the following features:

- stoppointid_first
- actualtime_arr_stop_first
- stoppointid_next
- actualtime_arr_stop_next (the target feature)
- month
- day_of_week             
- weekday
- bank_holiday
- rain
- temp
- rhum
- msl

The following features have not been used for this initial model:

- dayofservice: the relevant information from this feature has been split into other features e.g. month, day_of_week
- tripid: we will not be able to provide this information to the model
- lineid: constant as the initial model just covers one route
- routeid: we may not be able to map this to the timetable information we have and if not won't be able to provide it to the model
- direction: could look at including in a future model
- progrnumber_first: probably shouldn't use as routes can change over time so progrnumber for a stop may not be the same as what the model was trained on
- progrnumber_next: as above
- day: could be included in a future model, but seems to have less correlation with other features compared to day_of_week
- hour: could be included in a future model
- record_date: the relevant information from this feature has been split into other features e.g. month, day_of_week
- wetb: not available from OpenWeather
- dewpt: not available from OpenWeather
- vappr: not available from OpenWeather
- time_diff: could look at including in a future model as target feature instead of actualtime_arr_stop_next.

# 13. Prepare Categorical Features

We will use binary encoding to avoid wrongly assigning an order to the categorical features where one isn't really present. The following features will be encoded:

- stoppointid_first
- stoppointid_next
- month
- day_of_week

**weekday** and **bank_holiday** are already binary so don't need to be encoded.

In [ ]:
# encode values for stoppointid_first
df_dummies = pd.get_dummies(df_merged['stoppointid_first'], prefix='stoppointid_first')
df_merged = pd.concat([df_merged, df_dummies], axis =1)

In [ ]:
# encode values for stoppointid_next
df_dummies = pd.get_dummies(df_merged['stoppointid_next'], prefix='stoppointid_next')
df_merged = pd.concat([df_merged, df_dummies], axis =1)

In [ ]:
# encode values for month
df_dummies = pd.get_dummies(df_merged['month'], prefix='month')
df_merged = pd.concat([df_merged, df_dummies], axis =1)

In [ ]:
# encode values for day_of_week
df_dummies = pd.get_dummies(df_merged['day_of_week'], prefix='day_of_week')
df_merged = pd.concat([df_merged, df_dummies], axis =1)

# Appendix 1: Data Quality Plan - Bus Data (Before Transformation)

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| tenderlot       | All rows are null | Drop feature |
| suppressed_trip | All rows are null | Drop feature |
| justificationid_trip | All rows are null | Drop feature |
| passengers | All rows are null | Drop feature | 
| passengersin | All rows are null | Drop feature |
| passengersout | All rows are null | Drop feature |
| distance_leavetimes | All rows are null | Drop feature |
| note_leavetimes | All rows are null | Drop feature |
| note_vehicle | All rows are null | Drop feature |
| datasource | Constant feature | Drop feature |
| lineid | Constant feature | This is constant because we just have data for one route loaded. At some point we may process more than one route together so will keep feature for now. May not be needed to train the model. |
| basin | Constant feature | Drop feature |
| lastupdate_trip | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| note_trip | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| suppressed_leavetimes | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| justifcationid_leavetimes | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| lastupdate_leavetimes | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| vehicleid | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| distance_vehicle | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| minutes_vehicle | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| actualtime_arr_trip | Missing values < 1% | Ignore for now as this feature is not brought across when data is transformed. |
| actualtime_dep_trip | Missing values < 3% | Ignore for now as this feature is not brought across when data is transformed. |

# Appendix 2: Data Quality Plan - Bus Data (After Transformation)

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| stoppointid_first | Missing values ~ 1% | Drop affected rows |
| actualtime_arr_stop_first | Missing values ~ 1%| Drop affected rows |
| stoppointid_next | Missing values ~ 1% | Drop affected rows |
| actualtime_arr_stop_next | Missing values ~ 1%| Drop affected rows |

# Appendix 3: Data Quality Plan - Weather Data

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| itemp | Constant feature | Drop feature |
| iwb | Constant feature | Drop feature |
| rain | Missing data - 2 rows | Imputation - replace with 0 after looking at data for other timestamps on the same day |
| irain | Seems to be a missing data indicator | Drop feature as only two rows have missing data, and imputation is performed for these rows. |

# Appendix 4: Data Quality Plan - After Merge & Features Added

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| time_diff | For 167 rows, this feature is zero or negative | Drop affected rows|
| All | Missing data for segment 1354 to 7578 due to stop 7589 no longer being on the route. | Decide on an approach - other routes may have data for this segment. If not, we may need to use timetable information or real time info to estimate travel time through this segment.  There is also the possibility of using times between 1354 and 7589 and 7589 and 7578 to approximate travel time, but we would need more info on where stop 7589 was to see if this would be valid to do. |

# Appendix 5: Tests for *transform_data*

In [ ]:
df_test1 = df.loc[5:100]
df_test1 = df_test1.reset_index(drop=True)
df_test1

In [ ]:
df_transformed1 = transform.transform_data(df_test1)
df_transformed1

In [ ]:
pieces = [df[:35], df[42:100]]
df_test2 = pd.concat(pieces)
df_test2 = df_test2.reset_index(drop=True)
df_test2

In [ ]:
df_transformed2 = transform.transform_data(df_test2)
df_transformed2

In [ ]:
pieces = [df[:5], df[10:100]]
df_test3 = pd.concat(pieces)
df_test3 = df_test3.reset_index(drop=True)
df_test3

In [ ]:
df_transformed3 = transform.transform_data(df_test3)
df_transformed3

In [ ]:
pieces = [df[:5], df[8:10], df[14:50]]
df_test4 = pd.concat(pieces)
df_test4 = df_test4.reset_index(drop=True)
df_test4

In [ ]:
df_transformed4 = transform.transform_data(df_test4)
df_transformed4